In [36]:
#!/usr/bin/env python
import sys
from ome_model.experimental import Plate, Image, create_companion
import subprocess
import pandas as pd


#!/usr/bin/env python
from ome_model.experimental import Plate, Image, create_companion
import subprocess
import re

plate_name = 'AU00902'
columns = ['1']
rows = ['A']
fields = ['1']
timepoints = ['00d00h00m', '04d00h00m']
channels = ['G', 'R', 'P']
PATTERN = re.compile(r"(\d{2})d(\d{2})h(\d{2})m")
print(f"Creating {plate_name}.companion.ome ...")

plate = Plate(plate_name, len(rows), len(columns))
for row_index, row in enumerate(rows):
    for column_index, column in enumerate(columns):
        well = plate.add_well(row_index, column_index)
        for field_index, field in enumerate(fields):
            # Create multi-channel timelapse image per field of view
            image_name = f"{plate_name}_{row}{column}_{field}"
            image = Image(
                image_name, 1408, 1040, 1, len(channels), len(timepoints),
                order="XYZTC", type="uint16")
            # Create channels
            image.add_channel(
                name='G',
                color = int.from_bytes([0, 255, 0, 255], 'big', signed=True),
                samplesPerPixel=1)
            image.add_channel(
                name='R',
                color = int.from_bytes([255, 0, 0, 255], 'big', signed=True),
                samplesPerPixel=1)
            image.add_channel(
                name='P',
                color = int.from_bytes([255, 255, 255, 255], 'big', signed=True),
                samplesPerPixel=1)
            tiff_folder = f"{plate_name}/{row}{column}/field_{field}/"
            for t, timepoint in enumerate(timepoints):
                # Parse timestamp from filename
                m = PATTERN.match(timepoint)
                t_min = (
                    int(m.group(1)) * 60 * 24 +
                    int(m.group(2)) * 60 +
                    int(m.group(3)))
                for c, channel in enumerate(channels):
                    # Define TIFF files for each plane
                    tiff_filename = (f"{plate_name}_{channel}_{row}{column}"
                                     f"_{field}_{timepoint}.tif")
                    image.add_tiff(tiff_folder + tiff_filename, c=c, z=0, t=t)
                    # Add plane metadata for each plane with timestamp
                    options = {
                        'DeltaT': f"{t_min}",
                        'DeltaTUnit': 'min',
                    }
                    image.add_plane(c=c, z=0, t=t, options=options)
            well.add_wellsample(field_index, image)

companion_file = f"{plate_name}.companion.ome"
create_companion(plates=[plate], out=companion_file)

# Indent XML for readability
proc = subprocess.Popen(
    ['xmllint', '--format', '-o', companion_file, companion_file],
    stdin=subprocess.PIPE,
    stdout=subprocess.PIPE)
(output, error_output) = proc.communicate()

print("Done.")


In [37]:
data_path = "AU565_library.csv"
plate_name = "AU00901"

#filenames = []
#with open(data_path) as fp:
#    line = fp.readline()
#    while line:
#        filenames.append(line.strip())
#        line = fp.readline()


In [54]:
#{‘a’: np.float64, ‘b’: np.int32, ‘c’: ‘Int64’}
file_metadata = pd.read_csv(data_path, dtype={'column': 'str'})
#file_metadata = file_metadata[file_metadata['plateID']==plate_name]
file_metadata = file_metadata.iloc[0:2] #delete after debug



In [53]:
#plate_name = sys.argv[1]
plate_name = 'AU00901'

time_points = pd.unique(file_metadata['timepoint'])


plate = Plate(plate_name, len(file_metadata['row']), len(file_metadata['column']))

def test_func(x):
    print(x['filename'])
    basename = "{}{}{}".format(x['row'], x['column'], x['field'])
    image = Image(basename, 2080, 1552, 3, len(timepoints),
                          order="XYCT", type="uint16")
            image.add_channel(samplesPerPixel=3)
    return()

bar = file_metadata.apply(test_func, axis = 1)
        

IndentationError: unexpected indent (<ipython-input-53-fee72bcbc815>, line 15)

In [ ]:

print("Creating {}.companion.ome ...".format(plate_name))
plate = Plate(plate_name, len(rows), len(columns))
well_index = 0
for row_index, row in enumerate(rows):
    for column_index, column in enumerate(columns):
        well = plate.add_well(row_index, column_index)
        test = "{}_{}{}_0h.tiff".format(plate_name, row, column)
        if test in filenames:
            basename = "{}{}".format(row, column)
            image = Image(basename, 2080, 1552, 25, 3, len(timepoints),
                          order="XYZTC", type="uint8")
            image.add_channel(samplesPerPixel=3)
            for i, timepoint in enumerate(timepoints):
                filename = "{}_{}{}_{}.tiff".format(plate_name, row, column, timepoint)
                image.add_tiff(filename, c=0, z=0, t=i, planeCount=25)
                options = {
                    'DeltaT': timepoint[:-1],
                    'DeltaTUnit': 'h',
                }
                for z in range(25):
                    image.add_plane(c=0, z=z, t=i, options=options)
            well.add_wellsample(well_index, image)
            well_index += 1
companion_file = "../companions/{}.companion.ome".format(plate_name)
create_companion(plates=[plate], out=companion_file)

# Indent XML for readability
proc = subprocess.Popen(
    ['xmllint', '--format', '-o', companion_file, companion_file],
    stdin=subprocess.PIPE,
    stdout=subprocess.PIPE)
(output, error_output) = proc.communicate()

print("Done.")